In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
sales = pd.DataFrame({
    'id': [101, 102, 103, 104, 105, 106],
    'client_id': [1, 2, 1, 3, 4, 5],  # client_id = 4 and 5 not in clients
    'item': ['TechBook Pro 15', 'ViewMax 24HD', 'KeyPro MX', 'TechBook Air', 'ClickOne Mini', 'SoundX Pro'],
    'amount': [1200, 300, 100, 1300, 50, 500]
})

clients = pd.DataFrame({
    'client_id': [1, 2, 3, 6],  # client_id = 6 is not in sales
    'client_name': ['Alice', 'Bob', 'Charlie', 'Diana'],
    'region_id': [10, 11, 10, 12]
})

In [3]:
#find users who placed an order but are absent in "clients":
#first, merge datasets with outer to see all values, including nans:
df = sales.merge(clients, on = 'client_id', how = 'outer')
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,client_id,item,amount,client_name,region_id
0,101.0,1,TechBook Pro 15,1200.0,Alice,10.0
1,103.0,1,KeyPro MX,100.0,Alice,10.0
2,102.0,2,ViewMax 24HD,300.0,Bob,11.0
3,104.0,3,TechBook Air,1300.0,Charlie,10.0
4,105.0,4,ClickOne Mini,50.0,NaN,NaN
5,106.0,5,SoundX Pro,500.0,NaN,NaN
6,NaN,6,NaN,NaN,Diana,12.0


In [4]:
#now from the resulting dataframe, find users who placed an order but are absent in "clients":
absent_clients = df.loc[df['client_name'].isna(), 'client_id'].tolist()
print(f"Ids of clients who placed order but are absent in the client list are: {absent_clients}")

Ids of clients who placed order but are absent in the client list are: [4, 5]


In [5]:
#find how many user didn't place order:
no_order_clients = len(df.loc[df['item'].isna()])
print(f"The number of clients who didn't place order is: {no_order_clients}")

The number of clients who didn't place order is: 1


In [6]:
#replace nans in "client_name" with "Unknown client"
#replace nans in "item" with "No order"
df["client_name"] = df["client_name"].fillna("Unknown client") #use fillna
df["item"] = np.where(df["item"].isna(), "No order", df["item"]) #use np.where()
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,client_id,item,amount,client_name,region_id
0,101.0,1,TechBook Pro 15,1200.0,Alice,10.0
1,103.0,1,KeyPro MX,100.0,Alice,10.0
2,102.0,2,ViewMax 24HD,300.0,Bob,11.0
3,104.0,3,TechBook Air,1300.0,Charlie,10.0
4,105.0,4,ClickOne Mini,50.0,Unknown client,NaN
5,106.0,5,SoundX Pro,500.0,Unknown client,NaN
6,NaN,6,No order,NaN,Diana,12.0


In [7]:
#show the each client's number of orders:
#this is tricky because "No order" is just another item for the program and it'll count it as such.
#to solve this problem, group clients with itesm, create boolean mask on "No order", where item != No order is True, item == No order is False
#for each group, sum up True(1) and False(0)
number_of_orders = df.groupby('client_name')['item'].apply(lambda x: (x != 'No order').sum())
number_of_orders = pd.DataFrame(number_of_orders).reset_index()
number_of_orders.columns = ['Client', 'Number_of_orders']
number_of_orders.sort_values(by = 'Number_of_orders', ascending = False)

,Client,Number_of_orders
0,Alice,2
4,Unknown client,2
1,Bob,1
2,Charlie,1
3,Diana,0
